* https://www.kaggle.com/aleksandradeis/globalwheatdetection-eda

* https://www.kaggle.com/shonenkov/training-efficientdet

In [ ]:
import os
import cv2
import re
import random
import numpy as np
import pandas as pd

# For Visualization
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# For Image Augmentation
import albumentations as A
from PIL import Image,ImageDraw
from ast import literal_eval

# For parallel processing
from joblib import Parallel,delayed

# For Object Detection
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [ ]:
def load_model(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [ ]:
os.chdir('/kaggle/input/global-wheat-detection')
os.listdir()

In [ ]:
test=os.listdir('test')

In [ ]:
train=pd.read_csv('train.csv')

In [ ]:
train.head()

In [ ]:
train['x_min']=train['bbox'].apply(lambda x:float(re.findall(r'[0-9.]+',x.split(',')[0])[0]))
train['y_min']=train['bbox'].apply(lambda x:float(re.findall(r'[0-9.]+',x.split(',')[1])[0]))
train['box_width']=train['bbox'].apply(lambda x:float(re.findall(r'[0-9.]+',x.split(',')[2])[0]))
train['box_height']=train['bbox'].apply(lambda x:float(re.findall(r'[0-9.]+',x.split(',')[3])[0]))

In [ ]:
train.drop('bbox',axis=1,inplace=True)

In [ ]:
train['box_area']=train['box_width']*train['box_height']
train['x_max']=train['x_min']+train['box_width']
train['y_max']=train['y_min']+train['box_height']

In [ ]:
print(f'No of Unique Images are {train.image_id.nunique()}')

# Visualizing Images

In [ ]:
def show_images(df,rows,cols,title,linecolor):
    """ Function to show images with detected objects"""
    fig, axs = plt.subplots(rows, cols, figsize=(10,10))
    
    for row in range(rows):
        for col in range(cols):
            idx = np.random.randint(len(df), size=1)[0]
            images=df.iloc[idx].image_id

            path=os.path.join('train',images+'.jpg')
            image=Image.open(path)

            objects=train[train['image_id']==images][['x_min','x_max','y_min','y_max']].values
            # Drawing on the Image
            draw=ImageDraw.Draw(image)

            for box in objects:
                draw.rectangle([box[0],box[2],box[1],box[3]],width=10,outline=linecolor)
            plt.figure(figsize=(10,10))
            axs[row,col].imshow(image)
            axs[row,col].axis('off')
    plt.suptitle(title)

In [ ]:
# No of Bounding Boxes per Image
train['count']=train.apply(lambda row: 1 if np.isfinite(row.width) else 0, axis=1)
train_images_spikes= train.groupby('image_id')['count'].sum().reset_index()

In [ ]:
# Visualizing Spikes
counts,bins=np.histogram(train_images_spikes['count'],bins=range(0,117,4))
labels=[]
for i in range(0,len(bins)-1):
    labels.append(f'({bins[i]}-{bins[i+1]}]')
px.bar(x=labels,y=counts,labels={'x':'Range of Wheats per Images','y':'count'},title="Number of wheat spikes per image")

In [ ]:
# Visualizing Images with Higher spikes
show_images(train_images_spikes[train_images_spikes['count']>50],3,3,'Images with Higher Spikes','red')

In [ ]:
sns.distplot(train['box_area'],kde=False,bins=100)

In [ ]:
show_images(train[train['box_area']>500000],3,3,'Images with large bounding box','red')

 So we need to delete boxes with large areas

In [ ]:
image_transforms=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                            [0.229,0.224,0.225])
])

In [ ]:
# Spliting the dataset
train_set,val_set=train_test_split(train,test_size=0.15,random_state=1)

In [ ]:
class DatasetGenerator(Dataset):
    
    def __init__(self,dataset,transforms=None,test=False):
        super().__init__()
        self.dataset=dataset # Train dataset which contains boxes
        self.transforms=transforms # Image Transformation
        self.test=test # Test set
    
    def __getitem__(self,index):
        image_id = self.dataset.iloc[index]['image_id']
        img_path=os.path.join(f'train/{image_id}.jpg')
        img=Image.open(img_path)
        
        # Convert everything into a torch tensor
        boxes=torch.tensor(self.dataset[self.dataset['image_id']==image_id]
                           [['x_min','y_min','x_max','y_max']].values,dtype=torch.float32)
        
        # Classes
        labels=torch.ones((boxes.shape[0]),dtype=torch.int64)
        iscrowd=torch.zeros((boxes.shape[0],), dtype=torch.int64)        
        target={}
        
        target['boxes']=boxes
        target['labels']=labels
        target['image_id']=torch.tensor([index])
        target['area']=torch.tensor(self.dataset[self.dataset['image_id']==image_id]['box_area'].values,
                                    dtype=torch.float32)
        target['iscrowd']=iscrowd
        
        if self.transforms is not None:
            img=self.transforms(img)
        return img,target
    
    def __len__(self):
        return self.dataset['image_id'].nunique()

In [ ]:
train_dataset = DatasetGenerator(
    dataset=train_set,
    transforms=image_transforms,
    test=False,
)

validation_dataset = DatasetGenerator(
    dataset=val_set,
    transforms=image_transforms,
    test=True,
)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader=DataLoader(train_dataset,batch_size=4,shuffle=False,num_workers=4,collate_fn=collate_fn)
test_loader=DataLoader(validation_dataset,batch_size=4,shuffle=False,num_workers=4,collate_fn=collate_fn)

# Loading Pretrained FasterRCNN Model

In [ ]:
DEVICE=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
NUM_EPOCHS=1
LR=0.005

In [ ]:
# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (Wheat spikes) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Train

In [ ]:
model=model.to(DEVICE)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
train_loss=0
val_loss=0
epochs=list(range(NUM_EPOCHS))

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=LR,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
loss_hist = Averager()
itr = 1

for epoch in range(NUM_EPOCHS):
    loss_hist.reset()
    
    for images, targets in train_loader:
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        del images,targets

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}") 


In [ ]:
os.chdir('/kaggle/working')

In [ ]:
checkpoint={'model':model,
           'state_dict':model.state_dict()}


torch.save(checkpoint, 'fasterrcnn_resnet50_fpn.pth')

In [ ]:
"""model=load_model('fasterrcnn_resnet50_fpn.pth')

model=model.to(DEVICE)

box=[]
with torch.no_grad():
    for img_name in test:
        img_path=os.path.join(f'test/{img_name}')
        img=Image.open(img_path)

        img=image_transforms(img)
        img=img.unsqueeze(0)

        img=img.to(DEVICE)

        output=model(img)
        output=output[0]['boxes'].cpu().detach().flatten().tolist()
        boxes=''
        for i in output:
            boxes=boxes+str(i)+' '
        box.append(boxes)
        del img,boxes,output,img_path

prediction=pd.DataFrame({'image_id':test,'PredictionString':box})

os.chdir('/kaggle/working')

#prediction.to_csv('submission.csv',index=False)"""